In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm

In [3]:
file = open('MobiAct_Dataset_v2.0/Readme.txt', 'r', encoding='latin1')
strings = file.readlines()
file.close()

In [4]:
person_list, activity_list = [], []
for s in strings:
    if '|' in s:
        temp = s.split('|')
        temp = [x.strip() for x in temp]
        if 'sub' in s and len(temp) == 9:
            person_list.append(temp[3:-1])
        if len(temp) == 8:
            activity_list.append(temp[1:-1])

In [5]:
falls = ['FOL', 'FKL', 'BSC', 'SDL']

columns = ['name', 'age', 'height', 'weight', 'gender']
person_info = pd.DataFrame(person_list, columns=columns)

activity_info = pd.DataFrame(activity_list)
activity_info.columns = activity_info.iloc[0]
activity_info = activity_info.drop([0, 13])
activity_info = activity_info.reset_index(drop=True)
index = activity_info['No.']
activity_info = activity_info.drop(['No.'], axis=1)
activity_info.index = index
activity_info['label_encoded'] = list(range(len(activity_info)))

In [9]:
activity_info.to_csv('mobiact_preprocessed/activity_info.csv')
person_info.to_csv('mobiact_preprocessed/person_info.csv')

In [6]:
data_dir = 'MobiAct_Dataset_v2.0/Annotated Data/'
act_list = os.listdir(data_dir)
print(act_list)

['CHU', 'FOL', 'CSI', 'JOG', 'CSO', 'SBE', 'WAL', 'SBW', 'SLW', 'SIT', 'SCH', 'FKL', 'SRH', 'STD', 'BSC', 'JUM', 'SDL', 'SLH', 'STN', 'STU']


In [8]:
import shutil

save_dir = 'mobiact_preprocessed/'

train_dir, valid_dir, test_dir = 'train/', 'valid/', 'test/'

for dir_name in [train_dir, valid_dir, test_dir]:
    if not os.path.exists(save_dir + dir_name):
        os.makedirs(save_dir + dir_name)
for act in act_list:
    file_dir = data_dir + act + '/'
    file_list = os.listdir(file_dir)
    for file in file_list:
        person_num = int(file.split('_')[1])
        if person_num <= 50:
            shutil.move(file_dir + file, save_dir + train_dir)
        elif person_num > 50 and person_num < 57:
            shutil.move(file_dir + file, save_dir + valid_dir)
        elif person_num >= 57:
            shutil.move(file_dir + file, save_dir + test_dir)